In [158]:
import boto3
import json
import base64
import time
import re
import os
import logging
import redis
import os
from datetime import datetime
from botocore.config import Config
from botocore.exceptions import ClientError
config = Config(read_timeout=1000)

# Bedrock Runtime client initialization
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name= os.environ.get("Region"), #"us-east-1",
    config=config
)

# Bedrock client initialization
bedrock = boto3.client(
    service_name='bedrock',
    region_name= os.environ.get("Region") #'us-east-1'
)
# Initialize Redis client
redis_host = 'ch-agent-assist-redis-cluster-disabled.tjyhst.ng.0001.use1.cache.amazonaws.com'
redis_port = 6379  # Default Redis port is 6379
redis_password = None  # None if no password
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)

##Processing data to fetch role and content
def construct_call_conversation(data):
    segments = json.loads(data)
    convo = "" 
    # Extract transcripts, participant roles, and content
    for segment in segments:
        transcript = segment['transcript'][0]
        participant_role = transcript['ParticipantRole']
        content = transcript['Content']
        convo += participant_role + " : " + content + "\n"
        
    return convo

def model_output_postprocessing(data):
    result = ""
    start_index = data.find("{")
    
    if start_index == -1:
        data1 = "{" + data
    else:
        data1 = data
    
    start_index_final = data.find("{")    
    print(f'start_index:{start_index_final}')
    
    end_char_indices = [i.start() for i in re.finditer("}",data1)]
    print(f'end_indices:{end_char_indices}')
    end_index = end_char_indices[len(end_char_indices)-1]
    print(f'end_index:{end_index}')
    if end_index == len(data1)-1:
        result = data1[start_index_final:]
    else:
        result = data1[start_index_final:end_index+1]
    print(f'result:{result}')
    
    return result
    
def summarisation_prompt_generator(context):
    prompt_llama = f"""
Instruction: "Summarise this call transcript between a patient and an agent and include the information shared by the patient in a precise paragraph":

Addtionally, use bullet points(*) and highlight the next steps mentioned in the transcript under a separate heading.

Do not give redundant information.
Do not create any headings for Summary.

NOTE: Consider the below context as your only source of information and do not create any additional information or context

{context}
             
             
Response :

    """        
    return prompt_llama

def get_summary_prompt(bucket,file,prompt_category,required_prompt,conversation):
    s3 = boto3.client('s3') 
    response = s3.get_object(Bucket=bucket,Key=file)
    content = response['Body'].read().decode('utf-8')
    json_content = json.loads(content)
    prompt = json_content[prompt_category][required_prompt].format(context=conversation)
    
    return prompt

#Defining function to summarize context
def load_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt" : prompt,
            "temperature" : temp,
            "top_p" : top_p,
            "max_gen_len" : 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logging.error("Couldn't invoke Llama 2")
        raise  

data ='''
"AGENT" : "Welcome to patient bad with this is there on March? Who do I have the pleasure of speaking with today?"

"CUSTOMER" : "Hi. This is Michael Gonzalez."

"AGENT" : "Same here. How can I help you today?"

"CUSTOMER" : "I wanted to inquire about enrolling in the patient service program."

"AGENT" : "Great. Uh, so I would need a few insurance details from you. But before that, Could you please confirm your gender and date of birth for me?"

"CUSTOMER" : "Sure. I'm female and born on November 19th. 1985"

"AGENT" : "Great. Could you please confirm your address for me?"

"CUSTOMER" : "It is 14098. Marlin coat. Mesa. Arizona."

"AGENT" : "And the Zip code B 85201."

"CUSTOMER" : "Actually, no, we are at 85296."

"AGENT" : "Okay, Great. Uh, Is this phone number 402818969 the correct number."

"CUSTOMER" : "Yes, This is the best phone number."

"AGENT" : "Okay. Could you please confirm your email ID for me?"

"CUSTOMER" : "My email is m I A g 23 at gmail dot com."

"AGENT" : "Great. Uh, yeah. Could you please help me with your insurance provider and, uh, your preferred pharmacy?"

"CUSTOMER" : "My insurance is with United Health. And my preferred pharmacy would be CVS."

"AGENT" : "Grade. Thank you. Let me do a final check."

"AGENT" : "Mm hmm. Great. Okay. Looks like I have all the information that is needed."

"AGENT" : "I am routing a consent form your way, how would you prefer to receive this form? text message or email?"

"AGENT" : "Once you fill in the consent form and submit, we can process the further processes"

"CUSTOMER" : "sure, text message is fine."

"AGENT" : "And here it goes, would you like to finish this process now when we are on the call?"

"CUSTOMER" : "yes, definitely, give me few mins here"

"AGENT" : "sure, take your time. I am here if you have any queries"

"CUSTOMER" : "Ah, I just submitted the form"

"AGENT" : "thank you, just allow me a couple of mins for it to populate it in the system"

"AGENT" : "and here we go! Got evrything we need , so your enrollment process is complete"

"CUSTOMER" : "that's great"

"AGENT" : "Thank you, our team is going to reach out over an email in the next 48 hours"

'''


# Generate summary prompt
#prompt_summary = get_summary_prompt(prompt_bucket, file_key, 'text_summarization', 'summary_llama2', final_transcript)
prompt_summary = summarisation_prompt_generator(data)
# Load Llama2 model
model_invoke_timestamp = datetime.now()
summary_data = load_llama2(bedrock_runtime, prompt_summary, 0.5, 0.9)
model_invoke_after_timestamp = datetime.now()
print(f'summary_data : lambda_handler--output_from_the_llm_model : {summary_data}.')
 


summary_data : lambda_handler--output_from_the_llm_model : 
    Summary:
    The customer, Michael Gonzalez, called to inquire about enrolling in the patient service program. The agent confirmed the customer's details such as gender, date of birth, address, and phone number. The customer provided their insurance information and preferred pharmacy. The agent then sent a consent form to the customer via text message, which they submitted and completed the enrollment process. The agent confirmed that the enrollment process is complete, and the customer will receive an email from the team in the next 48 hours.
    
    Next Steps:
    • The team will reach out to the customer via email in the next 48 hours.
    • The customer will receive a text message with the consent form.
    • The customer completed the enrollment process..


In [162]:
import boto3
import json
import base64
import time
import re
import os
import logging
import redis
import os
from datetime import datetime
from botocore.config import Config
from botocore.exceptions import ClientError
config = Config(read_timeout=1000)

# Bedrock Runtime client initialization
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name= os.environ.get("Region"), #"us-east-1",
    config=config
)

# Bedrock client initialization
bedrock = boto3.client(
    service_name='bedrock',
    region_name= os.environ.get("Region") #'us-east-1'
)
# Initialize Redis client
redis_host = 'ch-agent-assist-redis-cluster-disabled.tjyhst.ng.0001.use1.cache.amazonaws.com'
redis_port = 6379  # Default Redis port is 6379
redis_password = None  # None if no password
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)

##Processing data to fetch role and content
def construct_call_conversation(data):
    segments = json.loads(data)
    convo = "" 
    # Extract transcripts, participant roles, and content
    for segment in segments:
        transcript = segment['transcript'][0]
        participant_role = transcript['ParticipantRole']
        content = transcript['Content']
        convo += participant_role + " : " + content + "\n"
        
    return convo

def model_output_postprocessing(data):
    result = ""
    start_index = data.find("{")
    
    if start_index == -1:
        data1 = "{" + data
    else:
        data1 = data
    
    start_index_final = data.find("{")    
    print(f'start_index:{start_index_final}')
    
    end_char_indices = [i.start() for i in re.finditer("}",data1)]
    print(f'end_indices:{end_char_indices}')
    end_index = end_char_indices[len(end_char_indices)-1]
    print(f'end_index:{end_index}')
    if end_index == len(data1)-1:
        result = data1[start_index_final:]
    else:
        result = data1[start_index_final:end_index+1]
    print(f'result:{result}')
    
    return result
    
def summarisation_prompt_generator(context):
    prompt_llama = f"""
Instruction: "Create a content to be sent as an email to be sent to the patient post the call, which contains any follow-up tasks or requests with spoken actions/documents 


{context}
             
             
Response :

    """        
    return prompt_llama

def get_summary_prompt(bucket,file,prompt_category,required_prompt,conversation):
    s3 = boto3.client('s3') 
    response = s3.get_object(Bucket=bucket,Key=file)
    content = response['Body'].read().decode('utf-8')
    json_content = json.loads(content)
    prompt = json_content[prompt_category][required_prompt].format(context=conversation)
    
    return prompt

#Defining function to summarize context
def load_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt" : prompt,
            "temperature" : temp,
            "top_p" : top_p,
            "max_gen_len" : 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logging.error("Couldn't invoke Llama 2")
        raise  

data ='''
"AGENT" : "Welcome to patient bad with this is there on March? Who do I have the pleasure of speaking with today?"

"CUSTOMER" : "Hi. This is Michael Gonzalez."

"AGENT" : "Same here. How can I help you today?"

"CUSTOMER" : "I wanted to inquire about enrolling in the patient service program."

"AGENT" : "Great. Uh, so I would need a few insurance details from you. But before that, Could you please confirm your gender and date of birth for me?"

"CUSTOMER" : "Sure. I'm female and born on November 19th. 1985"

"AGENT" : "Great. Could you please confirm your address for me?"

"CUSTOMER" : "It is 14098. Marlin coat. Mesa. Arizona."

"AGENT" : "And the Zip code B 85201."

"CUSTOMER" : "Actually, no, we are at 85296."

"AGENT" : "Okay, Great. Uh, Is this phone number 402818969 the correct number."

"CUSTOMER" : "Yes, This is the best phone number."

"AGENT" : "Okay. Could you please confirm your email ID for me?"

"CUSTOMER" : "My email is m I A g 23 at gmail dot com."

"AGENT" : "Great. Uh, yeah. Could you please help me with your insurance provider and, uh, your preferred pharmacy?"

"CUSTOMER" : "My insurance is with United Health. And my preferred pharmacy would be CVS."

"AGENT" : "Grade. Thank you. Let me do a final check."

"AGENT" : "Mm hmm. Great. Okay. Looks like I have all the information that is needed."

"AGENT" : "I am routing a consent form your way, how would you prefer to receive this form? text message or email?"

"AGENT" : "Once you fill in the consent form and submit, we can process the further processes"

"CUSTOMER" : "sure, text message is fine."

"AGENT" : "And here it goes, would you like to finish this process now when we are on the call?"

"CUSTOMER" : "yes, definitely, give me few mins here"

"AGENT" : "sure, take your time. I am here if you have any queries"

"CUSTOMER" : "Ah, I just submitted the form"

"AGENT" : "thank you, just allow me a couple of mins for it to populate it in the system"

"AGENT" : "and here we go! Got evrything we need , so your enrollment process is complete"

"CUSTOMER" : "that's great"

"AGENT" : "Thank you, our team is going to reach out over an email in the next 48 hours"

'''


# Generate summary prompt
#prompt_summary = get_summary_prompt(prompt_bucket, file_key, 'text_summarization', 'summary_llama2', final_transcript)
prompt_summary = summarisation_prompt_generator(data)
# Load Llama2 model
model_invoke_timestamp = datetime.now()
summary_data = load_llama2(bedrock_runtime, prompt_summary, 0.5, 0.9)
model_invoke_after_timestamp = datetime.now()
print(f'summary_data : lambda_handler--output_from_the_llm_model : {summary_data}.')
 


summary_data : lambda_handler--output_from_the_llm_model : 












































































































































































































































































































































































































































































































































































































































































































































































































































































































































































